# Neptune Workshop Cleanup - Lab 4 (Optional)
This notebook walks through deleting all of the resources created by the Neptune Labs in this workshop. You should only need to perform these steps if you have deployed  the Retail Demo Store in your own AWS account and want to deprovision Neptune resources. If you are participating in an AWS-led workshop, this process is likely not necessary.

## Setup
Provision any resources required for cleanup.

### Import Dependencies

In [ ]:
import boto3
import time
import requests
import pandas as pd

### Create Clients
We will use boto3 to delete the resources that have been created.

In [ ]:
servicediscovery = boto3.client('servicediscovery')
ssm = boto3.client('ssm')
dynamodb = boto3.resource('dynamodb')
neptune = boto3.client('neptune')
ec2 = boto3.client('ec2')

### Load Saved Variables
Load variables saved throughout the labs containing information required to target resources for deletion.

In [ ]:
%store -r

## Delete Resources
Resources need to be deleted in a specific order to avoid dependency errors. In order, we will delete the Neptune database instance, the Neptune cluster, the subnet group, the parameter group and the security group used to connect to Neptune.

In [ ]:
# Delete DB instance
response = neptune.delete_db_instance(
    DBInstanceIdentifier=db_instance_identifier,
    SkipFinalSnapshot=True,
)
# Wait for deletion
status = None
max_time = time.time() + 15*60
while time.time() < max_time:
    try:
        response = neptune.describe_db_instances(DBInstanceIdentifier=db_instance_identifier)
    except:
        print('Deletion successful')
        break

    status = response['DBInstances'][0]['DBInstanceStatus']
    print("Status: {}".format(status))

    time.sleep(30)

In [ ]:
# Delete DB cluster
response = neptune.delete_db_cluster(
    DBClusterIdentifier=db_cluster_identifier,
    SkipFinalSnapshot=True,
)
# Wait for deletion
status = None
max_time = time.time() + 15*60
while time.time() < max_time:
    try:
        response = neptune.describe_db_clusters(DBClusterIdentifier=db_cluster_identifier)
    except:
        print('Deletion successful')
        break

    status = response['DBClusters'][0]['Status']
    print("Status: {}".format(status))

    time.sleep(30)

In [ ]:
# Delete subnet group
neptune.delete_db_subnet_group(
    DBSubnetGroupName=db_subnet_group_name
)
# Delete parameter group
neptune.delete_db_cluster_parameter_group(
    DBClusterParameterGroupName=parameter_group_name
)
# Delete security group
ec2.delete_security_group(
    GroupId=security_group_id
)
# Delete key pair
ec2.delete_key_pair(
    KeyName=keypair_name
)

### Delete Stacks
The `neptune-index` stack created in Lab 3 can be deleted from the CloudFormation console.

## Reset Resources
Some resources (DynamoDB and OpenSearch) were not created specifically for this workshop, but modifications were made over the labs. To revert these changes we have to reset the resources to the state they were in before the workshop.

### DynamoDB
We added a new attribute to the products table in DynamoDB to store the labels generated by Rekognition. This can be removed from all items.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)
assert len(response['Instances']) > 0, 'Products service instance not found; check ECS to ensure it launched cleanly'

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
response = requests.get('http://{}/products/all'.format(products_service_instance))
products = response.json()
products_df = pd.DataFrame(products)

ddb_table = dynamodb.Table(table_name['Parameter']['Value'])

for product in products_df.itertuples():
    # Update in DynamoDB
    ddb_table.update_item(
        Key={'id': product.id},
        UpdateExpression="REMOVE image_labels",
)

### OpenSearch
The OpenSearch index was updated to contain the data stored in Neptune. To reset the index, complete the [OpenSearch workshop](../0-StartHere/Search.ipynb).

## Cleanup Complete
All resources create by the Neptune workshop have been deleted.